# Aggregate 
Following on from "rt_daily_aggregations" (which, for RTD, has been slimmed down to processing data), aggregates observations

In [3]:
import pandas as pd
import dask.dataframe as dd
import boto3

In [4]:
""" Assemble pb to parquet dataframes, one per day """
import sys
from google.protobuf.json_format import MessageToJson
import json
from google.transit import gtfs_realtime_pb2
from io import BytesIO
import boto3
import pandas as pd
import pendulum
import dask.dataframe as dd

In [ ]:
# Step 

## Step 1
Create a dask dataframe from parquet files.
For development, add method to GTFSdata . Update combine_daily_files later

In [6]:
import sys
sys.path
sys.path.append('c:\\Users\\Chris\\CodingProjects\\rtd-ghost-buses\\')
# from  scrape_data.parse_data import gtfs_decode
from scrape_data.combine_daily_files import GTFSdata, VehiclePosition, TripUpdate, Alerts

In [8]:
def generate_dask_dataframe(self):
    s3_prefix = f"processed/{self.data_type}/*.parquet"
    

In [11]:
s3_prefix = f"processed/TripUpdate/*.parquet"
s3_path = f"s3://rtd-ghost-buses-private/{s3_prefix}"
s3_path

's3://rtd-ghost-buses-private/processed/TripUpdate/*.parquet'

In [14]:
dask_path = dd.read_parquet(s3_path)
dask_path.head()

,id,tripUpdate.trip.tripId,tripUpdate.trip.scheduleRelationship,tripUpdate.trip.routeId,tripUpdate.trip.directionId,tripUpdate.stopTimeUpdate,tripUpdate.vehicle.id,tripUpdate.vehicle.label,header.timestamp,filename
0,1672556583_114253564,114253564,CANCELED,101E,0,None,None,None,2023-01-01 07:03:03,bus_data_TripUpdate/2023-01-01T00:03:12.767143...
0,1672556583_114253883,114253883,CANCELED,101E,1,None,None,None,2023-01-01 07:03:03,bus_data_TripUpdate/2023-01-01T00:03:12.767143...
0,1672556583_114253884,114253884,CANCELED,101E,1,None,None,None,2023-01-01 07:03:03,bus_data_TripUpdate/2023-01-01T00:03:12.767143...
0,1672556583_114254175,114254175,CANCELED,101H,0,None,None,None,2023-01-01 07:03:03,bus_data_TripUpdate/2023-01-01T00:03:12.767143...
0,1672556583_114254183,114254183,CANCELED,101H,0,None,None,None,2023-01-01 07:03:03,bus_data_TripUpdate/2023-01-01T00:03:12.767143...


In [ ]:
# TODO - Aggregate! How?

In [9]:
GTFSdata.generate_dask_dataframe = generate_dask_dataframe